In [1]:
import SimpleITK as sitk
import numpy as np
import os

In [2]:
#help(sitk.WriteImage)

## Image Preprocessing:

In [3]:
def image_preprocessing(image, ref_image):
    
    """
    Apply N4BiasFieldCorrection, Histogram Matching according to a reference image,
    Image Z-normalization (mu = 0 and variance = 1), and 8-bit rescaling (0,255) over
    the input MRI image.
    
    Argument:
    image -- SimpleItk supported MRI image (.nift, .nrrd, ...), with pixel type sitk.Float32, that
    will be corrected;
    ref-image -- SimpleItk supported MRI image (.nift, .nrrd, ...), with pixel type sitk.Float32, that
    was already N4BiasCorrected;
    
    Returns:
    prep_image -- the MRI image after this preprocessing pipeline;
    """
    print ("Running image_preprocessing() routine.")
    print (" ")
    
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    
    corrected_image = corrector.Execute(image)

    print ("N4BiasCorrection Done!")
    print (" ")
    
    hist_matcher = sitk.HistogramMatchingImageFilter()

    matched_image = hist_matcher.Execute(corrected_image, ref_image)
    
    print ("Histogram Matching Done!")
    print (" ")
    
    z_normalizer = sitk.NormalizeImageFilter()

    z_normalized_image = z_normalizer.Execute(matched_image)
    
    print ("Image Normalization Done!")
    print (" ")
    
    rescaler = sitk.RescaleIntensityImageFilter()
    rescaler.SetOutputMinimum(0)
    rescaler.SetOutputMaximum(255)    
    
    rescaled_image = rescaler.Execute(z_normalized_image) 
    
    print ("Image rescaling Done!")
    print (" ")
    
    prep_image = rescaled_image
    
    return prep_image    

In [4]:
def make_images(group, image_type):
    
    """
    Read the images and labels files in folders and run feature extraction pipeline for a single patient
    lesion label. Since loading image, preprocessing, features extraction, features preprocessing,
    to final feature DataFrame for each greoup (MICCAI or WHM) and image_type (3DT1 or FLAIR).
    
    Argument:
    group -- WHM for Cerebrovascular patients, WHM Challenge patients, or MICCAI for Multiple Sclerosis, 
    MICCAI Challende Patients.
    
    Returns:
    final_dataframe - a DataFrame containing all extracted featuresfor both group accordint to image_type.
    There will be four csvs files at the end: WMH_3DT1, WMH_FLAIR, MICCAI_3DT1, MICCAI_FLAIR.
    """
    
    if group == 'MICCAI':
        patient_dir = '/home/leonardo/Documents/Projeto-Bizu-MS/MICCAI2016'
        image_3dt1 = '3DT1.nii.gz'
        image_flair = '3DFLAIR.nii.gz'
        prep_image_flair = 'prep_FLAIR.nrrd'
        prep_image_3dt1 = 'prep_3DT1.nrrd'

    if group == 'WMH':
        patient_dir = '/home/leonardo/Documents/Projeto-Bizu-MS/WHMChallenge'
        image_3dt1 = 'orig/3DT1.nii.gz'
        image_flair = 'orig/FLAIR.nii.gz'
        prep_image_flair = 'orig/prep_FLAIR.nrrd'
        prep_image_3dt1 = 'orig/prep_3DT1.nrrd'
     
    patient_count = 0

    patients = os.listdir(patient_dir)
    for patient in patients:
        
        print("Patient " + str(patient_count))
        print(' ')
        
        # Creating patient folder path
        patient_folder = os.path.join(patient_dir,patient)
        print(patient_folder)
        print(' ')

        if image_type == "FLAIR":
            image_file = os.path.join(patient_folder, image_flair)
            prep_image_name = os.path.join(patient_folder, prep_image_flair)
            ref_image_file = '/home/leonardo/Documents/Projeto-Bizu-MS/Classification-of-MRI-Hiperintense-Brain-Lesions/Image-Preprocessing-and-Radiomic-Extraction/HistogramMatchingReferenceImages/MICCAI-01016SACH-3FLAIR.nrrd'
        
        elif image_type == "3DT1":
            image_file = os.path.join(patient_folder, image_3dt1)
            prep_image_name = os.path.join(patient_folder, prep_image_3dt1)
            ref_image_file = '/home/leonardo/Documents/Projeto-Bizu-MS/Classification-of-MRI-Hiperintense-Brain-Lesions/Image-Preprocessing-and-Radiomic-Extraction/HistogramMatchingReferenceImages/MICCAI-01016SACH-3DT1.nrrd'
        
        if not os.path.isfile(image_file):
            print('OOOOOOOOOOOOOO - Images not found! - XXXXXXXXXXXX')
            print(' ')
            print(image_file)
            print(' ')
            
            continue
        
        print(image_file)
        print(' ')
        print(ref_image_file)
        print(' ')
        print(prep_image_name)
        print(' ')
        
        image = sitk.ReadImage(image_file, sitk.sitkFloat32)
        ref_image = sitk.ReadImage(ref_image_file, sitk.sitkFloat32)

        prep_image = image_preprocessing(image, ref_image)
        
        sitk.WriteImage(prep_image, prep_image_name)
        
        patient_count += 1
        
    return 

In [6]:
image_file = "/home/leonardo/Documents/Projeto-Bizu-MS/MICCAI2016/01016SACH/3DFLAIR.nii.gz"

image = sitk.ReadImage(image_file, sitk.sitkFloat32)

corrector = sitk.N4BiasFieldCorrectionImageFilter()
    
corrected_image = corrector.Execute(image)

sitk.WriteImage(corrected_image, '3DFlair.nrrd')


### Calling make images for both groups, both image types 

In [ ]:
make_images('WMH','3DT1')
make_images('WMH','FLAIR')

In [ ]:
make_images('MICCAI','3DT1')
make_images('MICCAI','FLAIR')

Patient 0
 
/home/leonardo/Documents/Projeto-Bizu-MS/MICCAI2016/08027SYBR
 
/home/leonardo/Documents/Projeto-Bizu-MS/MICCAI2016/08027SYBR/3DT1.nii.gz
 
/home/leonardo/Documents/Projeto-Bizu-MS/Classification-of-MRI-Hiperintense-Brain-Lesions/Image-Preprocessing-and-Radiomic-Extraction/HistogramMatchingReferenceImages/MICCAI-01016SACH-3DT1.nrrd
 
/home/leonardo/Documents/Projeto-Bizu-MS/MICCAI2016/08027SYBR/prep_3DT1.nrrd
 
Running image_preprocessing() routine.
 


## Image preparation

In [4]:
def label_resampler(image, reference_image):
    """
    Resample image into the reference_image physical space (origing, spacing, direction).
    At the end image will have same voxrl size as reference_image.
    
    Argument:
    image -- SimpleItk supported binary image (.nift, .nrrd, ...) that
    will be resampled;
    reference_image -- SimpleItk supported binary image (.nift, .nrrd, ...) that
    will be used as reference for origin, spacing, and direction; 
    
    Returns:
    resampled_image -- image with same physical space information as reference_image;
    """
    
    print ("Running label_resampler() routine.")
    print (" ")
    
    interpolator = sitk.sitkNearestNeighbor
    transform = sitk.Transform(3, sitk.sitkIdentity)
    resampled_image = sitk.Resample(image, reference_image, transform,
                              interpolator, 0.0, sitk.sitkUInt16)    
    
    return resampled_image    

In [5]:
def prepare_label(label_image, label_value):
    """
    Receives a binary image with zeros (background) and label_value intensities and
    change it into a binary image with zeros and ones where there was
    label_value intensities;
    
    Argument:
    label_image -- SimpleItk supported binary image (.nift, .nrrd, ...) with
    intensities: zeros (background) and label_values;
    label_value: integer that represents label information in the label_image; 
    
    Returns:
    one_zero_label -- a binary label image with ones and zeros(background)
    """
    print ("Running prepare_label() routine.")
    print (" ")
    
    one_zero_label_raw = sitk.Divide(label_image, int(label_value))
    one_zero_label = sitk.Cast(one_zero_label_raw, sitk.sitkUInt16)
    
    return one_zero_label

In [6]:
def crop_image_and_label(image, label_image, label_value):
    """
    Cropp a grey-level image and a label_image in the shortest box region that
    contains the label information in label_image;
    
    Argument:
    label_image -- SimpleItk supported binary image (.nift, .nrrd, ...) with
    intensities: zeros (background) and label_values that will be used to
    define the cropping box region and that will be cropped as well;
    image -- SimpleItk supported grey-level image (.nift, .nrrd, ...) that
    will be cropped;
    label_value: integer that represents label information in the label_image; 
    
    Returns:
    cropped_image -- the grey-level cropped image 
    cropped_label -- a binary label image with ones and zeros(background)
    """
    
    print ("Running crop_image_and_label() routine.")
    print (" ")
    
    one_zero_label = prepare_label(label_image, label_value)
    image_casted = sitk.Cast(image, sitk.sitkUInt16)
    only_roi_image = sitk.Multiply(one_zero_label,image_casted)
    
    shape = sitk.LabelShapeStatisticsImageFilter()
    shape.Execute(one_zero_label)
    region = shape.GetRegion(1)
    region_index = (region[0], region[1], region[2])
    region_size = (region[3], region[4], region[5])
    
    cropped_image = sitk.RegionOfInterest(only_roi_image, region_size, region_index)
    cropped_label = sitk.RegionOfInterest(one_zero_label, region_size, region_index)
    
    return (cropped_image, cropped_label)

In [7]:
def is_out_of_bounds(image_size, index):
    """
    Check if a gien pixel index is not inside image region, if it is out of bounds;
    
    Argument:
    image_size -- tuple with x, y, z image sizes;
    index -- 2D or 3D tuple that identifies the pixel to be checked; 
    
    Returns:
    True -- if index is out of bounds, False, otherwise; 
    """  
    
    if index[0] >= image_size[0] or index[0] < 0:
        return True
    elif index[1] >= image_size[1] or index[1] < 0:
        return True
    elif index[2] >= image_size[2] or index[2] < 0:
        return True
    else:
        return False

In [8]:
def get_neighborhood_mean(image, label_image, pixel_index):
    """
    Calculates the average grey-level intensity of a given pixel's  neighborhood
    (3x3x3 in 3D or 3x3 in 2D);
    
    Argument:
    label_image -- SimpleItk supported binary image (.nift, .nrrd, ...) with
    intensities: zeros (background) and ones;
    image -- SimpleItk supported grey-level image (.nift, .nrrd, ...);
    pixel_index: 3D or 3D tuple that identifies the pixel will have the
    average grey-level intensity calculated for its neighborhood; 
    
    Returns:
    neighborhood_mean -- pixel_index neighborhood average intensity 
    """    
    size = label_image.GetSize()
    
    idx = pixel_index
    n_sum = 0
    counted_pixel = 0
    for x in range(idx[0]-1,idx[0]+2):
        for y in range(idx[1]-1,idx[1]+2):
            for z in range(idx[2]-1,idx[2]+2):
                index = (x,y,z)
                if not is_out_of_bounds(size, index):
                    label_value = label_image.GetPixel(index)
                    if label_value == 1:
                        n_sum += image.GetPixel(index)
                        counted_pixel += 1

    neighborhood_mean = int(n_sum/counted_pixel)
      
    return neighborhood_mean

In [9]:
def outlier_removal(image, label_image):
    """
    Remove outlier pixel intensities by changing outlier values for
    the pixels neighborhood average intnsity. A pixel intensity is considered
    outlier when it is > mean + 3*std or < mean - 3*std;
    
    Argument:
    label_image -- SimpleItk supported binary image (.nift, .nrrd, ...) with
    intensities: zeros (background) and ones;
    image -- SimpleItk supported grey-level image (.nift, .nrrd, ...);
    
    Returns:
    no_outlier_image -- SimpleItk supported grey-level image (.nift, .nrrd, ...)
    with o putliers; 
    """
    print ("Running outlier_remova() routine.")
    print (" ")
    
    # cleaned final image
    no_outlier_image = image
    
    # Getting grey-level statistics
    statistics = sitk.LabelIntensityStatisticsImageFilter()
    statistics.Execute(label_image, image)
    mean = statistics.GetMean(1)
    std = statistics.GetStandardDeviation(1)
    
    size = image.GetSize()    
    for x in range(size[0]):
        for y in range(size[1]):
            for z in range(size[2]):
                index = (x,y,z)
                label_value = label_image.GetPixel(index)
                if label_value == 1:
                    image_value = image.GetPixel(index)
                    if (image_value > (mean + 3*std)) or (image_value < (mean - 3*std)):
                        # print("Outlier found: "+ str(image_value))                        
                        neighbourhood_mean = get_neighborhood_mean(image, label_image, index) 
                        # print("New value: "+ str(neighbourhood_mean))
                        no_outlier_image.SetPixel(x, y, z, neighbourhood_mean)               
    
    return no_outlier_image

## Reading and Preparation Pipeline Call 

In [10]:
def get_image_and_label_ready(image_file, label_file, label_value, ref_image_file):
    
    """
    Runs the whole image preprocessing pipeline.
    
    Argument:
    image_file -- dir that conttans a SimpleItk supported greylevel image (.nift, .nrrd, ...);
    label_file -- dir that conttans a SimpleItk supported binary image (.nift, .nrrd, ...);
    label_value -- integer that indicates the pixel value on label_image;
    ref_image_file -- dir that conttans a SimpleItk supported greylevel image (.nift, .nrrd, ...) 
    used as referencen in the histogram matching stage;
    
    Returns:
    final_image -- the post processed SimpleItk supported greylevel image (.nift, .nrrd, ...).
    This image contain only the greylevel lesion, outlier-filtered and rescaled;
    final_label -- the post processed SimpleItk supported binary image (.nift, .nrrd, ...).
    This label does ocuppy the same greylevel image physical space and is cropped containing
    only the label lesion region;
    """
    
    print ("Running get_image_and_label_ready() routine.")
    print (" ")
    
    label_image = sitk.ReadImage(label_file, sitk.sitkInt8)
    image = sitk.ReadImage(image_file, sitk.sitkFloat32)
    ref_image = sitk.ReadImage(ref_image_file, sitk.sitkFloat32)
    
    # resampling label image into 3DT1 space:
    resampled_label_image = label_resampler(label_image, image)
    
    prep_image = image_preprocessing(image, ref_image)
    
    cropped_image, cropped_label = crop_image_and_label(prep_image, resampled_label_image, label_value)
    
    no_outlier_image = outlier_removal(cropped_image, cropped_label)

    final_image, final_label = no_outlier_image, cropped_label
  
    return (final_image, final_label)

## RADIOMICS Routine: Perform feature extraction for each prepared image and label

In [5]:
import pandas as pd
import radiomics
import six
import os
import sys
import logging

### Configuring and extracting features

In [12]:
def extract_features(image, label):
    
    """
    Configure the setting and run the whole radiomics extraction pipeline.
    
    Argument:
    image -- a SimpleItk supported greylevel image (.nift, .nrrd, ...);
    label -- a SimpleItk supported binary image (.nift, .nrrd, ...);
    
    Returns:
    features -- list containing all the features name
    values -- list containing all the feature values for the respective label and value
    """
    
    print ("Running extract_features() routine.")
    print (" ")
    
    #setting up logger:
    # Get the PyRadiomics logger (default log-level = INFO)
    logger = radiomics.logger
    logger.setLevel(logging.DEBUG)  # set level to DEBUG to include debug log messages in log file

    # Set up the handler to write out all log entries to a file
    handler = logging.FileHandler(filename='testLog.txt', mode='w')
    formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)

    # Define settings for signature calculation
    settings = {}
    settings['binWidth'] = 3
    settings['label'] = 1
    
    # Initialize feature extractor by settings file (No other configuration must be set)
    extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(**settings)

    # By default, only original is enabled. Optionally enable some image types:
    extractor.enableImageTypeByName('Wavelet')
    extractor.enableImageTypeByName('LBP3D')
    extractor.enableImageTypeByName('Gradient')
        
    # Disable all classes except firstorder
    extractor.disableAllFeatures()

    # Enable all features in firstorder
    extractor.enableFeatureClassByName('firstorder')
    extractor.enableFeatureClassByName('glcm')
    extractor.enableFeatureClassByName('glrlm')
    extractor.enableFeatureClassByName('glszm')
    extractor.enableFeatureClassByName('gldm')
    extractor.enableFeatureClassByName('ngtdm')
    extractor.enableFeatureClassByName('shape')

    # Calculating features
    featureVector = extractor.execute(image, label)

    # priting and storing features
    features = []
    values = []

    i = 0 # feature counter 
    for featureName in featureVector.keys():
        # the results comes with a 32 lines header. The first condition is to avoid storing those information;
        if (i >= 22): 
            features.append(featureName) 
            values.append(featureVector[featureName])
            print("Computed %s: %s" % (featureName, featureVector[featureName]))
        i += 1
    
    return (features, values)

### Function to average wavelet and lbp repeated first_order features into single first_order features 

In [13]:
def get_averaged_features(all_features, image_key, ref_features):
    
    """
    Average all the feature eextracted from the filter-generated images. In this case,
    Wavelet option creates 8 different images and extract first order features from each of such images.
    This function average those 8 image features into a single set of first order features.
    The same goes for Local Binary pattern filter that generate 3 images and extract first order features 
    from each of those images.
    
    Argument:
    All_features -- A DataFrame ("Features", "Values") with all radiomic features extracted
    (from original image and from filtered images);
    image_key -- type of filter (wavelet, local binary pattern ...);
    ref_features -- is a list with the features that will be averaged and kept. E.g. first_order features.
    In the original extraction it is extracted all types of features (first order, coocurrence matrix, 
    run length ...) from the wavelet filtered images. 
    
    Returns:
    mean_image_key_features -- DataFrame containing the averaged filtered features;
    """
    
    print ("Running get_averaged_features() routine.")
    print (" ")
    
    image_key_features = all_features[all_features['Features'].str.contains(image_key)].copy()
    
    feat_means = []
    for feat in ref_features:
        feat_df = image_key_features[image_key_features['Features'].str.endswith('_'+feat)]
        mean = feat_df['Values'].mean()
        
        assert not mean == None, "Not calculating " + str(image_key)
        
        feat_means.append(mean)
        
    mean_image_key_features = pd.DataFrame({'Features':ref_features, 'Values':feat_means})
    mean_image_key_features['Features'] = mean_image_key_features['Features'].map(lambda x: x.replace(x,(image_key + '_' + x)))
    
    return mean_image_key_features

In [14]:
def generate_final_feature_set(features, values, patient_id):
    
    """
    Generates the final from all the features extracted.
    separate original_image features, call wavelet and local inary pattern averaged features and
    combine those features in to a single DataFrame;
    
    Argument:
    features -- list with names of all radiomic features extracted;
    values -- list with all radiomic feature values extracted;
    patient_id: id of patient and lesion that will identify the extraction;
    
    Returns:
    final_set - a DataFrame containing original_image extracted features, wavelet and local
    binary pattern extracted features, and the patient_id;
    
    """
    
    print ("Running generate_final_feature_set() routine.")
    print (" ")
    
    # generating a DF with all the extracted features:
    all_features = pd.DataFrame({'Features': features, 'Values': values})
    
    # separating the features calculated over the original image
    original = all_features[all_features['Features'].str.contains('original')] 
    
    # store first order feature names to use in the filtered image features averaging:
    first_order = original[original['Features'].str.contains('_firstorder_')]
    first_order = first_order['Features'].apply(lambda x: x.replace('original_firstorder_',''))
    first_order = np.array(first_order)
    first_order
    
    # getting averaged first order features calculated over wavelet and local_binary_pattern images;
    wavelet = get_averaged_features(all_features, 'wavelet', first_order)
    lbp = get_averaged_features(all_features, 'lbp', first_order)
    gradient = get_averaged_features(all_features, 'gradient', first_order)
    
    # Gethering all features (original, first_order wavelet and first_order lbp into a single DF)
    # Adding Patient_ID at the top row
    final_set = pd.DataFrame({'Features':['Patient_ID'], 'Values':patient_id})
    final_set = final_set.append(original, ignore_index=True)
    final_set = final_set.append(wavelet, ignore_index=True)
    final_set = final_set.append(lbp, ignore_index=True)
    final_set = final_set.append(gradient, ignore_index=True)
    final_set
    
    return(final_set)    

In [15]:
def get_final_set(image_file, label_file, label_value, ref_image_file, patient_id, classification):
    
    """
    Run the whole feature extraction pipeline for a single patient. Since loading image, 
    preprocessing, features extraction, features preprocessing, to final feature DataFrame
    assempling.
    
    Argument:
    image_file -- Dir of a SimpleItk supported greylevel image (.nift, .nrrd, ...);
    label_file -- Dir of a SimpleItk supported binary image (.nift, .nrrd, ...);
    label_value -- integer that defines label pixel intensity on label image;
    resample -- tells if the label image needs resampling into grey-level image physical space;
    
    Returns:
    final_set - a DataFrame containing original_image extracted features, wavelet and local
    binary pattern extracted features, and the patient_id;
    """
    
    print ("Running get_final_set() routine.")
    print (" ")
    
    # getting preprocessed images:
    image, label = get_image_and_label_ready(image_file, label_file, label_value, ref_image_file)
    
    features, values = extract_features(image, label)
    final_set = generate_final_feature_set(features, values, patient_id)
    
    # appending number of pixels into the final set
    final_set = final_set.append({'Features':'class', 'Values': classification}, ignore_index=True)
    
    return (final_set)

## Feature DataSet Generation 

In [16]:
def run(group, image_type):
    
    """
    Read the images and labels files in folders and run feature extraction pipeline for a single patient
    lesion label. Since loading image, preprocessing, features extraction, features preprocessing,
    to final feature DataFrame for each greoup (MICCAI or WHM) and image_type (3DT1 or FLAIR).
    
    Argument:
    group -- WHM for Cerebrovascular patients, WHM Challenge patients, or MICCAI for Multiple Sclerosis, 
    MICCAI Challende Patients.
    
    Returns:
    final_dataframe - a DataFrame containing all extracted featuresfor both group accordint to image_type.
    There will be four csvs files at the end: WMH_3DT1, WMH_FLAIR, MICCAI_3DT1, MICCAI_FLAIR.
    """
        
    group = group
    image_type = image_type
    
    if group == 'MICCAI':
        patient_dir = '/home/leonardo/Documents/Projeto-Bizu-MS/MICCAI2016'
        image_3dt1 = '3DT1.nii.gz'
        image_flair = '3DFLAIR.nii.gz'
        label_base = 'Consensus'
        classification = 0

    if group == 'WMH':
        patient_dir = '/home/leonardo/Documents/Projeto-Bizu-MS/WHMChallenge'
        image_3dt1 = 'orig/3DT1.nii.gz'
        image_flair = 'orig/FLAIR.nii.gz'
        label_base = 'label'
        classification = 1

    patient_count = 0
    label_count = 0

    data_frame = pd.DataFrame() 

    patients = os.listdir(patient_dir)
    for patient in patients:

        # Creating patient folder path
        patient_folder = os.path.join(patient_dir,patient)

        patient_count += 1

        if image_type == "FLAIR":
            image_file = os.path.join(patient_folder, image_flair)
            ref_image_file = '/home/leonardo/Documents/Projeto-Bizu-MS/Classification-of-MRI-Hiperintense-Brain-Lesions/Image Preprocessing and Radiomic Extraction/HistogramMatchingReferenceImages/MICCAI-01016SACH-3FLAIR.nrrd'
        elif image_type == "3DT1":
            image_file = os.path.join(patient_folder, image_3dt1)
            ref_image_file = '/home/leonardo/Documents/Projeto-Bizu-MS/Classification-of-MRI-Hiperintense-Brain-Lesions/Image Preprocessing and Radiomic Extraction/HistogramMatchingReferenceImages/MICCAI-01016SACH-3DT1.nrrd'
        
        if not os.path.isfile(image_file):
            print('Images not found!!!!!!!')
            print(image_file)
            print(' ')
            continue

        print(image_file)
        print(' ')

        # setting label_folder:
        label_folder = os.path.join(patient_folder,'labels')

        if not os.path.isdir(label_folder):
            print("No Label folder!")
            print(' ')
            continue

        # listing labels in label_folder
        labels = os.listdir(label_folder)    
        for label in labels:

            # Creating label folder path
            label_file = os.path.join(label_folder, label)      

            label_value = ''.join(list(filter(lambda x: x.isdigit(), label)))

            patient_id = group + '_' + patient + '_' + image_type + '_' + label_value
            print(patient_id)
            print(' ')

            patient_lesion_i = get_final_set(image_file, label_file, label_value, 
                                           ref_image_file, patient_id, classification)

            data_frame = data_frame.append(patient_lesion_i['Values'], ignore_index=True)

    print(patient_count, label_count)
    print(' ')
    data_frame.columns = patient_lesion_i['Features']
    
    csv_name = group + "_"+ image_type + ".csv"
    data_frame.to_csv(csv_name)
    
    return (data_frame)

In [ ]:
WMH_3DT1.to_csv(index=False)
WMH_FLAIR.to_csv(index=False)
MICCAI_3DT1.to_csv(index=False)
MICCAI_FLAIR.to_csv(index=False)